## Dataset

In [7]:
import pandas as pd
import numpy as np

### NYT Data

In [8]:
data1 = pd.read_csv('2013.07.02-2017.01.01.csv')
data2 = pd.read_csv('API_20170102_20170531.csv')
data3 = pd.read_csv('API_20170601_20171031.csv')
data4 = pd.read_csv('API_20171101_20171231.csv')
data5 = pd.read_csv('API_20180101_20180531.csv')
data6 = pd.read_csv('API_20180601_20181031.csv')
data7 = pd.read_csv('API_20181101_20181231.csv')
data8 = pd.read_csv('API_20190101_20190531.csv')
data9 = pd.read_csv('API_20190601_20191031.csv')
data10 = pd.read_csv('API_20191101_20191231.csv')
data11 = pd.read_csv('20100101-20130701.csv')

In [9]:
data2=data2.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
data3=data3.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
data4=data4.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
data5=data5.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)
data6=data6.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1)

In [10]:
data1.head()

,pub_date,lead_paragraph,main
0,2013-09-07T02:02:22+0000,"LONDON — Wilfred Owen, the British soldier-poe...","A Weapon Seen as Too Horrible, Even in War"
1,2013-09-16T02:52:31+0000,WASHINGTON — The Russian-American deal to dest...,"Brief Respite for President, but No Plan B on ..."
2,2013-10-31T01:43:52+0000,WASHINGTON — It was not obvious to the Nationa...,Tap on Merkel Provides Peek at Vast Spy Net
3,2013-11-09T02:13:13+0000,"However you look at it, it’s a rather static l...",Era Fades for Helping Hand at the Washroom Sink
4,2013-12-18T18:38:45+0000,WASHINGTON — A panel of outside advisers urged...,Obama Is Urged to Sharply Curb N.S.A. Data Mining


In [11]:
# Concat all data
data = pd.concat([data11, data1, data2, data3, data4, data5, data6, data7, data8, data9, data10])

In [12]:
data[['Date','The Rest']] = data['pub_date'].str.split("T", expand=True)
data.head()

,pub_date,lead_paragraph,main,Date,The Rest
0,2010-04-21T01:09:28+0000,"LAHORE, Pakistan — The professor was working i...","At Top University, a Fight for Pakistan’s Future",2010-04-21,01:09:28+0000
1,2010-02-24T02:29:35+0000,WASHINGTON  Tempers were fraying in the White...,Gentle White House Nudges Test the Power of Pe...,2010-02-24,02:29:35+0000
2,2010-01-12T01:52:19+0000,A law adopting statewide high school exams for...,"As School Exit Tests Prove Tough, States Ease ...",2010-01-12,01:52:19+0000
3,2010-03-03T04:05:42+0000,HOUSTON — Senator Kay Bailey Hutchison had all...,‘Yes’ for Texas Governor Is ‘No’ to Washington,2010-03-03,04:05:42+0000
4,2010-06-11T16:41:42+0000,WASHINGTON  Hired in 2001 by the National Sec...,Obama Takes a Hard Line Against Leaks to Press,2010-06-11,16:41:42+0000


In [13]:
data=data.drop(['pub_date', 'The Rest'], axis=1)

In [14]:
data.head()

,lead_paragraph,main,Date
0,"LAHORE, Pakistan — The professor was working i...","At Top University, a Fight for Pakistan’s Future",2010-04-21
1,WASHINGTON  Tempers were fraying in the White...,Gentle White House Nudges Test the Power of Pe...,2010-02-24
2,A law adopting statewide high school exams for...,"As School Exit Tests Prove Tough, States Ease ...",2010-01-12
3,HOUSTON — Senator Kay Bailey Hutchison had all...,‘Yes’ for Texas Governor Is ‘No’ to Washington,2010-03-03
4,WASHINGTON  Hired in 2001 by the National Sec...,Obama Takes a Hard Line Against Leaks to Press,2010-06-11


In [ ]:
companies = {
    'AAPL': ['AAPL', 'Apple', 'Tim Cook', 'iPhone'],
    'AMZN': ['AMZN', 'Amazon', 'Jeff Bezos'],
    'BRK-B': ['BRK-B', 'Berkshire Hathaway', 'Warren Buffett'],
    'GOOG': ['GOOG', 'Google', 'Sundar Pichai', 'Alphabet'],
    'JNJ': ['JNJ', 'Johnson & Johnson', 'harmac'],
    'META': ['META', 'Meta', 'Facebook', 'Mark Zuckerberg'],
    'MSFT': ['MSFT', 'Microsoft', 'Windows'],
    'NVDA': ['NVDA', 'NVIDIA', 'Jensen Huang', 'GPU'],
    'TSLA': ['TSLA', 'Tesla', 'Elon Musk'],
    'V': ['Visa', 'redit card']
}

for company, keywords in companies.items():
    data[company] = np.where((data['lead_paragraph'].str.contains('|'.join(keywords))) | (data['main'].str.contains('|'.join(keywords))), company, 0)

### Finbert

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [16]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [18]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Using device:", device)

Using device: cuda


In [19]:
# Define function to get sentiment scores for a batch of articles
def get_sentiments_batch(texts):
    # Tokenize the articles
    tokens = tokenizer.batch_encode_plus(texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        predictions = model.forward(**tokens)
    # Get the sentiment scores
    sentiment_scores = predictions.logits
    return sentiment_scores

# Get the articles and concatenate the main and lead paragraphs
data['text'] = (data['main'] + data['lead_paragraph']).astype('str')
texts = data[['Date', 'text']]

In [20]:
# Batch process the articles and get the sentiment scores
batch_size = 64
num_batches = (len(texts) + batch_size - 1) // batch_size
sentiment_scores = []
for i in range(num_batches):
    batch_texts = texts['text'].iloc[i*batch_size:(i+1)*batch_size].tolist()
    batch_scores = get_sentiments_batch(batch_texts)
    sentiment_scores.append(batch_scores.cpu())

In [21]:
# Concatenate the sentiment scores and convert to dataframe
sentiment_scores = torch.cat(sentiment_scores, dim=0).numpy()
sentiments_df = pd.DataFrame(sentiment_scores, columns=['Positive', 'Negative', 'Neutral'])
sentiments_df['Date'] = texts['Date'].values
sentiments_df = sentiments_df[['Date', 'Positive', 'Negative', 'Neutral']]

In [48]:
sentiments_df

,Date,Positive,Negative,Neutral
0,2010-04-21,-1.586529,1.963762,0.768027
1,2010-02-24,-1.384922,1.870671,0.572891
2,2010-01-12,-1.265072,1.767548,0.500417
3,2010-03-03,0.880638,-1.441588,0.133482
4,2010-06-11,-1.217777,1.404924,0.746163
...,...,...,...,...
21387,2019-12-01,-1.467058,1.398181,1.304486
21388,2019-11-18,-0.204274,-0.506046,1.250838
21389,2019-11-22,-0.696337,-1.079409,2.285552
21390,2019-12-03,-0.580811,-1.348498,2.333664


In [60]:
text_scores = texts.reset_index(drop=True)

In [64]:
text_scores['Positive'] = sentiments_df['Positive']
text_scores['Negative'] = sentiments_df['Negative']
text_scores['Neutral'] = sentiments_df['Neutral']
text_scores

,Date,text,Positive,Negative,Neutral
0,2010-04-21,"At Top University, a Fight for Pakistan’s Futu...",-1.586529,1.963762,0.768027
1,2010-02-24,Gentle White House Nudges Test the Power of Pe...,-1.384922,1.870671,0.572891
2,2010-01-12,"As School Exit Tests Prove Tough, States Ease ...",-1.265072,1.767548,0.500417
3,2010-03-03,‘Yes’ for Texas Governor Is ‘No’ to Washington...,0.880638,-1.441588,0.133482
4,2010-06-11,Obama Takes a Hard Line Against Leaks to Press...,-1.217777,1.404924,0.746163
...,...,...,...,...,...
21387,2019-12-01,How a Divided Left Is Losing the Battle on Abo...,-1.467058,1.398181,1.304486
21388,2019-11-18,"In Shift, U.S. Says Israeli Settlements in Wes...",-0.204274,-0.506046,1.250838
21389,2019-11-22,They Toil Gladly Offstage. Impeachment Lands T...,-0.696337,-1.079409,2.285552
21390,2019-12-03,A Welcome Funding Source for College Athletics...,-0.580811,-1.348498,2.333664


In [23]:
nyt_sentiments_d = sentiments_df.groupby('Date').agg({'Positive': 'mean', 'Negative': 'mean', 'Neutral': 'mean'}).reset_index()
nyt_sentiments_d

,Date,Positive,Negative,Neutral
0,2010-01-01,-0.223772,-0.265419,0.743379
1,2010-01-02,-0.774897,0.575262,0.730220
2,2010-01-03,-1.388011,2.037136,0.184805
3,2010-01-04,-0.585848,-0.055188,1.074774
4,2010-01-05,-0.771272,-0.427670,1.645177
...,...,...,...,...
3637,2019-12-27,-0.428388,-0.186135,1.132637
3638,2019-12-28,-1.270863,1.043080,1.048303
3639,2019-12-29,-1.167260,0.921415,1.224472
3640,2019-12-30,-1.009659,1.004473,0.774530


In [24]:
nyt_sentiments_d['Date'].equals(nyt_sentiments_d['Date'].sort_values())

True

In [25]:
macro_SAs = nyt_sentiments_d[['Date']].iloc[3:].reset_index(drop=True)

In [26]:
macro_SAs['Pos_lag1'] = nyt_sentiments_d['Positive'].iloc[2:-1].reset_index(drop=True)
macro_SAs['Pos_lag2'] = nyt_sentiments_d['Positive'].iloc[1:-2].reset_index(drop=True)
macro_SAs['Pos_lag3'] = nyt_sentiments_d['Positive'].iloc[:-3].reset_index(drop=True)

macro_SAs['Neg_lag1'] = nyt_sentiments_d['Negative'].iloc[2:-1].reset_index(drop=True)
macro_SAs['Neg_lag2'] = nyt_sentiments_d['Negative'].iloc[1:-2].reset_index(drop=True)
macro_SAs['Neg_lag3'] = nyt_sentiments_d['Negative'].iloc[:-3].reset_index(drop=True)

macro_SAs['Neu_lag1'] = nyt_sentiments_d['Neutral'].iloc[2:-1].reset_index(drop=True)
macro_SAs['Neu_lag2'] = nyt_sentiments_d['Neutral'].iloc[1:-2].reset_index(drop=True)
macro_SAs['Neu_lag3'] = nyt_sentiments_d['Neutral'].iloc[:-3].reset_index(drop=True)

In [34]:
macro_SA_W = nyt_sentiments_d[['Date']].iloc[7:].reset_index(drop=True)

In [35]:
macro_SA_W['Pos_lag1'] = nyt_sentiments_d['Positive'].iloc[6:-1].reset_index(drop=True)
macro_SA_W['Pos_lag2'] = nyt_sentiments_d['Positive'].iloc[5:-2].reset_index(drop=True)
macro_SA_W['Pos_lag3'] = nyt_sentiments_d['Positive'].iloc[4:-3].reset_index(drop=True)
macro_SA_W['Pos_lag4'] = nyt_sentiments_d['Positive'].iloc[3:-4].reset_index(drop=True)
macro_SA_W['Pos_lag5'] = nyt_sentiments_d['Positive'].iloc[2:-5].reset_index(drop=True)

macro_SA_W['Neg_lag1'] = nyt_sentiments_d['Negative'].iloc[6:-1].reset_index(drop=True)
macro_SA_W['Neg_lag2'] = nyt_sentiments_d['Negative'].iloc[5:-2].reset_index(drop=True)
macro_SA_W['Neg_lag3'] = nyt_sentiments_d['Negative'].iloc[4:-3].reset_index(drop=True)
macro_SA_W['Neg_lag4'] = nyt_sentiments_d['Negative'].iloc[3:-4].reset_index(drop=True)
macro_SA_W['Neg_lag5'] = nyt_sentiments_d['Negative'].iloc[2:-5].reset_index(drop=True)

macro_SA_W['Neu_lag1'] = nyt_sentiments_d['Neutral'].iloc[6:-1].reset_index(drop=True)
macro_SA_W['Neu_lag2'] = nyt_sentiments_d['Neutral'].iloc[5:-2].reset_index(drop=True)
macro_SA_W['Neu_lag3'] = nyt_sentiments_d['Neutral'].iloc[4:-3].reset_index(drop=True)
macro_SA_W['Neu_lag4'] = nyt_sentiments_d['Neutral'].iloc[3:-4].reset_index(drop=True)
macro_SA_W['Neu_lag5'] = nyt_sentiments_d['Neutral'].iloc[2:-5].reset_index(drop=True)

In [36]:
macro_SA_W

,Date,Pos_lag1,Pos_lag2,Pos_lag3,Pos_lag4,Pos_lag5,Neg_lag1,Neg_lag2,Neg_lag3,Neg_lag4,Neg_lag5,Neu_lag1,Neu_lag2,Neu_lag3,Neu_lag4,Neu_lag5
0,2010-01-08,-0.337798,-0.397189,-0.771272,-0.585848,-1.388011,-0.270438,-0.509768,-0.427670,-0.055188,2.037136,1.032809,1.311303,1.645177,1.074774,0.184805
1,2010-01-09,-1.285020,-0.337798,-0.397189,-0.771272,-0.585848,1.857027,-0.270438,-0.509768,-0.427670,-0.055188,0.039138,1.032809,1.311303,1.645177,1.074774
2,2010-01-10,-0.831077,-1.285020,-0.337798,-0.397189,-0.771272,0.623881,1.857027,-0.270438,-0.509768,-0.427670,0.882219,0.039138,1.032809,1.311303,1.645177
3,2010-01-11,-1.718464,-0.831077,-1.285020,-0.337798,-0.397189,2.308614,0.623881,1.857027,-0.270438,-0.509768,0.311725,0.882219,0.039138,1.032809,1.311303
4,2010-01-12,0.391811,-1.718464,-0.831077,-1.285020,-0.337798,-1.218776,2.308614,0.623881,1.857027,-0.270438,0.891924,0.311725,0.882219,0.039138,1.032809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,2019-12-27,-1.120693,-1.289652,-1.422672,-0.623928,-0.742815,0.822957,0.474469,1.298837,0.866436,-0.215681,1.079569,1.414802,0.922188,0.311617,1.655890
3631,2019-12-28,-0.428388,-1.120693,-1.289652,-1.422672,-0.623928,-0.186135,0.822957,0.474469,1.298837,0.866436,1.132637,1.079569,1.414802,0.922188,0.311617
3632,2019-12-29,-1.270863,-0.428388,-1.120693,-1.289652,-1.422672,1.043080,-0.186135,0.822957,0.474469,1.298837,1.048303,1.132637,1.079569,1.414802,0.922188
3633,2019-12-30,-1.167260,-1.270863,-0.428388,-1.120693,-1.289652,0.921415,1.043080,-0.186135,0.822957,0.474469,1.224472,1.048303,1.132637,1.079569,1.414802


In [44]:
dates = pd.read_csv('AAPL_w.csv')

In [45]:
dates

,Date,adjusted_close,volume,direction,Symbol
0,2020-01-31,75.6888,216776323,0,AAPL
1,2020-01-24,77.8408,115445527,0,AAPL
2,2020-01-17,77.9435,162824452,1,AAPL
3,2020-01-10,75.8894,168452059,1,AAPL
4,2020-01-03,72.7347,131852981,1,AAPL
...,...,...,...,...,...
521,2010-02-05,5.9407,131125200,1,AAPL
522,2010-01-29,5.8375,252672600,0,AAPL
523,2010-01-22,6.0103,101145800,0,AAPL
524,2010-01-15,6.2589,96055100,0,AAPL


In [46]:
# Filter macro_SAs with dates in daily
macro_SAs_filtered = macro_SA_W[macro_SA_W['Date'].isin(dates['Date'])]

# Filter daily with dates not in macro_SAs
dates_filtered = dates[~dates['Date'].isin(macro_SAs['Date'])]

# Print filtered dataframes
print("Macro_SAs filtered with daily dates:")
print(macro_SAs_filtered)
print("\nDaily filtered with dates not in macro_SAs:")
print(dates_filtered['Date'].tolist())

Macro_SAs filtered with daily dates:
            Date  Pos_lag1  Pos_lag2  Pos_lag3  Pos_lag4  Pos_lag5  Neg_lag1  \
0     2010-01-08 -0.337798 -0.397189 -0.771272 -0.585848 -1.388011 -0.270438   
7     2010-01-15 -1.112372 -1.157942 -1.295557  0.391811 -1.718464  1.240757   
14    2010-01-22 -0.917268 -0.613714 -0.896956 -0.613898 -1.309655  0.511129   
21    2010-01-29 -0.710976 -0.687080 -0.373176 -0.905567  1.278356  0.310767   
28    2010-02-05 -1.139397 -1.103406 -0.379929 -0.446723 -0.200701  1.527458   
...          ...       ...       ...       ...       ...       ...       ...   
3602  2019-11-29 -0.195285 -1.132883 -1.312470 -1.032862 -0.534254 -0.291682   
3609  2019-12-06 -1.203208 -1.218701 -1.099835 -0.788235 -0.941192  0.998612   
3616  2019-12-13 -0.745983 -0.471655 -0.568973 -1.276287 -0.850256  0.894252   
3623  2019-12-20 -0.974356 -0.390918 -1.334122 -1.021473 -0.463958  0.778836   
3630  2019-12-27 -1.120693 -1.289652 -1.422672 -0.623928 -0.742815  0.822957   

  

In [47]:
macro_SAs_filtered.to_csv('NYT_macro_SA_w.csv', index=False)

## Filter by Stock

In [88]:
companies = {
    'AAPL': ['AAPL', 'Apple', 'Tim Cook', 'iPhone'],
    'AMZN': ['AMZN', 'Amazon', 'Jeff Bezos'],
    'BRK-B': ['BRK-B', 'Berkshire Hathaway', 'Warren Buffett'],
    'GOOG': ['GOOG', 'Google', 'Sundar Pichai', 'Alphabet'],
    'JNJ': ['JNJ', 'Johnson & Johnson', 'harmac'],
    'META': ['META', 'Meta', 'Facebook', 'Mark Zuckerberg'],
    'MSFT': ['MSFT', 'Microsoft', 'Windows'],
    'NVDA': ['NVDA', 'Nvidia', 'Jensen Huang', 'GPU', 'conductor', 'rocessor'],
    'TSLA': ['TSLA', 'Tesla', 'Elon Musk', 'lectric car', 'lectric vehicle'],
    'V': ['Visa', 'redit card']
}

for company, keywords in companies.items():
    data[company] = np.where((data['lead_paragraph'].str.contains('|'.join(keywords))) | (data['main'].str.contains('|'.join(keywords))), company, 0)

In [89]:

# create an empty dictionary to hold the dataframes
dfs = {}

# iterate through the companies dict
for company, keywords in companies.items():
    # create a dataframe for the current company
    # filter the texts based on the keywords for the current company
    mask = pd.Series(text_scores['text'].str.contains('|'.join(keywords)))
    df = text_scores[mask]
    # add the dataframe to the dfs dictionary
    dfs[company] = df

In [90]:
[len(dfs[company]) for company in dfs]

[94, 85, 2, 98, 45, 191, 23, 7, 18, 39]